In [16]:
import numpy as np
import random as python_random
import tensorflow as tf
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(123)
# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error
from tensorflow.python.keras.models import load_model
from tensorflow.keras.initializers import Constant
from tensorflow import keras

In [2]:
'''
例1： model: call或者layer:call里面对一个batchadd_loss, add_metric相当于metric.update_state(y_batch, y_pred_batch)。
     但是add_loss会被自动重置，也就是每个batch结束后self.loss得到的结果就是这个batch的loss
        add_metric是累计的结果，也就是每个batch结束后self.metrics里面得到结果是累积的，需要手动重置（一般是一个epoch结束后）
'''
class LogisticEndpoint(keras.layers.Layer):
  def __init__(self, name=None):
    super(LogisticEndpoint, self).__init__(name=name)
    self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
    self.accuracy_fn = keras.metrics.BinaryAccuracy()

  def call(self, targets, logits, sample_weights=None):
    # Compute the training-time loss value and add it
    # to the layer using `self.add_loss()`.
    loss = self.loss_fn(targets, logits, sample_weights)
    self.add_loss(loss)

    # Log accuracy as a metric and add it
    # to the layer using `self.add_metric()`.
    acc = self.accuracy_fn(targets, logits, sample_weights)
    self.add_metric(acc, name="accuracy")

    # Return the inference-time prediction tensor (for `.predict()`).
    return tf.nn.softmax(logits)

In [5]:
targets = tf.ones((2, 1))
logits = tf.zeros((2, 1))
layer = LogisticEndpoint()
y = layer(targets, logits)
print("current accuracy value:", float(layer.metrics[0].result()))
print("current loss:", layer.losses)

current accuracy value: 0.0
current loss: [<tf.Tensor: shape=(), dtype=float32, numpy=0.6931472>]


In [6]:
targets = tf.ones((2, 1))
logits = tf.ones((2, 1))
y = layer(targets, logits)
print("current accuracy value:", float(layer.metrics[0].result()))
print("current loss:", layer.losses)

current accuracy value: 0.5
current loss: [<tf.Tensor: shape=(), dtype=float32, numpy=0.31326172>]


In [7]:
targets = tf.ones((2, 1))
logits = tf.ones((2, 1))
y = layer(targets, logits)
print("current accuracy value:", float(layer.metrics[0].result()))
print("current loss:", layer.losses)

current accuracy value: 0.6666666865348816
current loss: [<tf.Tensor: shape=(), dtype=float32, numpy=0.31326172>]


In [ ]:
# loss：无论是计算每个样本loss的底层函数,e.g.,keras.losses.binary_crossentropy
# 还是计算整个batch的高层函数，e.g.,tf.keras.losses.BinaryCrossentropy，都是每次调用后自动重置的
y_true = [[0], [0], [1], [0]]
y_pred = [[0.2], [0.5], [0.3], [0.3]]
bce = keras.losses.BinaryCrossentropy()
mse = keras.losses.MeanSquaredError()
print('bce_instance', bce(y_true, y_pred).numpy())
print('bce_instance', bce(y_true, y_pred).numpy())
print('bce_fn_reduce_mean', tf.reduce_mean(keras.losses.binary_crossentropy(y_true, y_pred)))
print('bce_fn_reduce_mean', tf.reduce_mean(keras.losses.binary_crossentropy(y_true, y_pred)))
print('mse_instance', mse(y_true, y_pred).numpy())
print('mse_instance', mse(y_true, y_pred).numpy())
print('mse_fn_reduce_mean', tf.reduce_mean(keras.losses.mean_squared_error(y_true, y_pred)))
print('mse_fn_reduce_mean', tf.reduce_mean(keras.losses.mean_squared_error(y_true, y_pred)))

In [55]:
# mask住一部分样本，计算损失函数
print('bce_fn_mask', keras.losses.binary_crossentropy(y_true, y_pred) * np.array([0, 0, 1, 1]))
# loss instance是对所有样本求平均
print('bce_instance_mask', bce(y_true[2:], y_pred[2:]).numpy())
# 如果利用sample_weight来mask的话，相当于只计算没被mask样本的loss，但是平均的时候还是对全部样本做平均
print('bce_instance_weight', bce(y_true, y_pred, sample_weight=[0, 0, 1, 1]).numpy())
print('bce_fn_mask_reduce_mean', tf.reduce_mean(keras.losses.binary_crossentropy(y_true, y_pred)* np.array([0, 0, 1, 1])))

# mask住一部分样本，计算损失函数
print('mse_fn_mask', keras.losses.mean_squared_error(y_true, y_pred) * np.array([0, 0, 1, 1]))
# loss instance是对所有样本求平均
print('mse_instance_mask', mse(y_true[2:], y_pred[2:]).numpy())
# 如果利用sample_weight来mask的话，相当于只计算没被mask样本的loss，但是平均的时候还是对全部样本做平均
print('mse_instance_weight', mse(y_true, y_pred, sample_weight=[0, 0, 1, 1]).numpy())
print('mse_fn_mask_reduce_mean', tf.reduce_mean(keras.losses.mean_squared_error(y_true, y_pred)* np.array([0, 0, 1, 1])))

bce_fn_mask tf.Tensor([0.        0.        1.2039725 0.3566748], shape=(4,), dtype=float32)
bce_instance_mask 0.7803236
bce_instance_weight 0.3901618
bce_fn_mask_reduce_mean tf.Tensor(0.3901618, shape=(), dtype=float32)
mse_fn_mask tf.Tensor([0.         0.         0.48999998 0.09      ], shape=(4,), dtype=float32)
mse_instance_mask 0.29
mse_instance_weight 0.145
mse_fn_mask_reduce_mean tf.Tensor(0.145, shape=(), dtype=float32)


In [56]:
# metric: update_state是累计的效应，需要手工reset_state
y_true = [[0], [0], [1], [0]]
y_pred = [[0.2], [0.5], [0.3], [0.3]]
auc = keras.metrics.AUC()
mse = keras.metrics.MeanSquaredError()
bce = keras.metrics.BinaryCrossentropy()
auc.update_state(y_true, y_pred, sample_weight=)
mse.update_state(y_true, y_pred)
bce.update_state(y_true, y_pred)
print('auc_instance', auc.result().numpy())
print('mse_instance', mse.result().numpy())
print('bce_instance', bce.result().numpy())
y_true_2 = [[1], [1], [0], [0]]
y_pred_2 = [[0.8], [0.5], [0.3], [0.1]]
auc.update_state(y_true_2, y_pred_2)
mse.update_state(y_true_2, y_pred_2)
bce.update_state(y_true_2, y_pred_2)
print(auc.result().numpy())
print(mse.result().numpy())
print(bce.result().numpy())
auc.reset_states()
mse.reset_states()
bce.reset_states()
auc.update_state(y_true, y_pred)
mse.update_state(y_true, y_pred)
bce.update_state(y_true, y_pred)
print('auc_instance', auc.result().numpy())
print('mse_instance', mse.result().numpy())
print('bce_instance', bce.result().numpy())

auc_instance 0.5
mse_instance 0.2175
bce_instance 0.61923444
0.8333333
0.1575
0.4819079
auc_instance 0.5
mse_instance 0.2175
bce_instance 0.61923444


In [57]:
# mask情况下的metric
auc.reset_states()
mse.reset_states()
bce.reset_states()
auc.update_state(y_true[2:], y_pred[2:])
mse.update_state(y_true[2:], y_pred[2:])
bce.update_state(y_true[2:], y_pred[2:])
print('auc_instance_mask', auc.result().numpy())
print('mse_instance_mask', mse.result().numpy())
print('bce_instance_mask', bce.result().numpy())
# 如果利用sample_weight来mask的话，相当于只计算没被mask样本的metric，平均的时候也是对没被mask的样本做平均
auc.reset_states()
mse.reset_states()
bce.reset_states()
auc.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
mse.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
bce.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
print('auc_instance_weight', auc.result().numpy())
print('auc_instance_weight', mse.result().numpy())
print('auc_instance_weight', bce.result().numpy())

auc_instance_mask 0.5
mse_instance_mask 0.29
bce_instance_mask 0.7803236
auc_instance_weight 0.5
auc_instance_weight 0.29
auc_instance_weight 0.7803236


In [10]:
# 综合来看metric:
# 如果每次update_state都是一个batch, 里面含有mask(使用weight实现)
# 不断update_state直到一个epoch结束以后的结果，就相当于把这个epoch里面不被mask的样本单独全拎出来做metric计算
y_true = [[0], [0], [1], [0]]
y_pred = [[0.2], [0.5], [0.3], [0.3]]
auc = keras.metrics.AUC()
mse = keras.metrics.MeanSquaredError()
bce = keras.metrics.BinaryCrossentropy()
auc.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
mse.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
bce.update_state(y_true, y_pred, sample_weight=np.array([0, 0, 1, 1]))
print('auc_instance', auc.result().numpy())
print('mse_instance', mse.result().numpy())
print('bce_instance', bce.result().numpy())
y_true2 = [[1], [1], [0], [0]]
y_pred2 = [[0.8], [0.5], [0.3], [0.1]]
auc.update_state(y_true2, y_pred2, sample_weight=np.array([0, 0, 1, 1]))
mse.update_state(y_true2, y_pred2, sample_weight=np.array([0, 0, 1, 1]))
bce.update_state(y_true2, y_pred2, sample_weight=np.array([0, 0, 1, 1]))
print('auc_instance', auc.result().numpy())
print('mse_instance', mse.result().numpy())
print('bce_instance', bce.result().numpy())
auc.reset_states()
mse.reset_states()
bce.reset_states()
auc.update_state(y_true+y_true2, y_pred+y_pred2, sample_weight=np.array([0, 0, 1, 1, 0, 0, 1, 1]))
mse.update_state(y_true+y_true2, y_pred+y_pred2, sample_weight=np.array([0, 0, 1, 1, 0, 0, 1, 1]))
bce.update_state(y_true+y_true2, y_pred+y_pred2, sample_weight=np.array([0, 0, 1, 1, 0, 0, 1, 1]))
print('auc_instance', auc.result().numpy())
print('mse_instance', mse.result().numpy())
print('bce_instance', bce.result().numpy())

auc_instance 0.5
mse_instance 0.29
bce_instance 0.7803236
auc_instance 0.6666666
mse_instance 0.17
bce_instance 0.5056706
auc_instance 0.6666666
mse_instance 0.17
bce_instance 0.5056706


In [69]:
# sklearn和keras里面的metric是一样的逻辑，mask的样本不参与分母部分
y_true = [[1], [1], [0], [0], [0], [1], [1], [0]]
y_pred = [[0.8], [0.5], [0.3], [0.1], [0.6], [0.7], [0.2], [0.2]]
auc_score = roc_auc_score(y_true, y_pred)
fpr, tpr, _ = roc_curve(y_true, y_pred)
ks = np.max(np.abs(tpr - fpr))
mse = mean_squared_error(y_true, y_pred)
print('sklearn_auc', auc_score)
print('sklearn_ks', ks)
print('sklearn_mse', mse)

auc_score = roc_auc_score(y_true[:-2], y_pred[:-2])
fpr, tpr, _ = roc_curve(y_true[:-2], y_pred[:-2])
ks = np.max(np.abs(tpr - fpr))
mse = mean_squared_error(y_true[:-2], y_pred[:-2])
print('sklearn_auc_mask', auc_score)
print('sklearn_ks_mask', ks)
print('sklearn_mse_mask', mse)

auc_score = roc_auc_score(y_true, y_pred, sample_weight=np.array([1, 1, 1, 1, 1, 1, 0, 0]))
fpr, tpr, _ = roc_curve(y_true, y_pred, sample_weight=np.array([1, 1, 1, 1, 1, 1, 0, 0]))
ks = np.max(np.abs(tpr - fpr))
mse = mean_squared_error(y_true, y_pred, sample_weight=np.array([1, 1, 1, 1, 1, 1, 0, 0]))
print('sklearn_auc_weight', auc_score)
print('sklearn_ks_weight', ks)
print('sklearn_mse_weight', mse)

sklearn_auc 0.78125
sklearn_ks 0.5
sklearn_mse 0.19
sklearn_auc_mask 0.888888888888889
sklearn_ks_mask 0.6666666666666667
sklearn_mse_mask 0.14
sklearn_auc_weight 0.888888888888889
sklearn_ks_weight 0.6666666666666667
sklearn_mse_weight 0.14


In [13]:
'''
例2：求导
   1. 常数需要手动watch才能求导，否则为None
   2. 如果y不依赖于z,则导数为None
   3. 如果需要反复求导，需要在tape里面加persist
'''

[None, None]
[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]
[<tf.Tensor: shape=(), dtype=float32, numpy=56.0>] [<tf.Tensor: shape=(), dtype=float32, numpy=3.0>]


In [37]:
x = tf.Variable(3.0)
z = tf.Variable(4.0)
y = tf.multiply(x, 2)
# x.assign(z)
with tf.GradientTape() as g:
  y = tf.identity(y)
dy_dx = g.gradient(y, [x, z])
print(dy_dx)

[None, None]


In [ ]:
x = tf.constant(3.0)
z = tf.constant(56.0)
with tf.GradientTape() as g:
  y = x * x
dy_dx = g.gradient(y, [x, z])
print(dy_dx)

In [ ]:
with tf.GradientTape() as g:
  g.watch(x)
  g.watch(z)
  y = x * x
dy_dx = g.gradient(y, [x, z])
print(dy_dx)

In [ ]:
with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  g.watch(z)
  y = x * z
dy_dx = g.gradient(y, [x])
dy_dz = g.gradient(y, [z])
print(dy_dx, dy_dz)

In [42]:
x = tf.Variable(2.0)
z = x * 3
with tf.GradientTape(persistent=True) as g:
  y = x * z
# 求导的分母，必须是能通过依赖关系推导得到底层的variable
# dy/dx: x是底层variable，所以可以进行求导。但是z的依赖关系在tape里面没有出现，因此dy/dx只能给到z, z=3*x=6
# dy/dz：z不是底层底层variable，而且它依赖关系在tape外，无法得到，因此求导为None
dy_dx = g.gradient(y, [x, z])
print(dy_dx)
# 求导的分子，必须是在tape里面定义的变量
# dz/dx: x是底层variale，但是z不是在taple定义的（虽然在tape里面的表达式里出现过），求导为None
dz_dx = g.gradient(z, [x])
print(dz_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]
[None]


In [41]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as g:
  z = x * 3
  y = x * z
# 求导的分母，必须是能通过依赖关系推导得到底层的variable
# dy/dx: x是底层variable，所以可以进行求导。并且z的依赖关系在tape里面是有的，因此dy/dx=d(x*x*3)/dx=6x=12
# dy/dz：z不是底层variable，但依赖关系在tape里面可以得到底层variable x，因此求导为x = 2
dy_dx = g.gradient(y, [x, z])
print(dy_dx)
# 求导的分子，必须是在tape里面定义的变量
# dz/dx: x是底层variale，z是在taple定义的，求导为3
dz_dx = g.gradient(z, [x])
print(dz_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=12.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=3.0>]


In [45]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as g:
  z = x * 3
  # 注意这里必须返回一个同名变量，否则相当于不起效果
  z = tf.stop_gradient(z)
  y = x * z

# 求导的分母，必须是能通过依赖关系推导得到底层的variable
# dy/dx: x是底层variable，所以可以进行求导。z的依赖关系在tape里面, 但是z不能对任何变量求导，因此dy/dx=z=3*x=6
# dy/dz：z不是底层variable，但依赖关系在tape里面可以得到底层variable x，因此求导为x = 2
dy_dx = g.gradient(y, [x, z])
print(dy_dx)
# 求导的分子，必须是在tape里面定义的变量
# dz/dx: x是底层variale，z是在taple定义的，但是z不能对任何变量求导，因此为None
dz_dx = g.gradient(z, [x])
print(dz_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]
[None]


In [2]:
x = tf.Variable(2.0)
f = []
with tf.GradientTape(persistent=True) as g:
  z = x * 3
  # 注意这里必须返回一个同名变量，否则相当于不起效果
  z = tf.stop_gradient(z)
  f.append(x * z)

# 求导的分子，必须是在tape里面定义过
# f[0]元素也是在tape里面定义的，因此也可以求导，效果同上
dy_dx = g.gradient(f[0], [x, z])
print(dy_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]
[None]


In [24]:
x = tf.Variable(2.0)
j = tf.Variable(2.0)
f = []
with tf.GradientTape(persistent=True) as g:
  z = x * 3
  f.append(z)
  k = x * 6
  f.append(k)
  # 得到shape = (2,)的tensor
  ff = tf.stack(f)
  # 得到shape = (3,2)的tensor
  ll = tf.stack([ff, ff, ff])
  kz = k / z

# 求导的分子，必须是在tape里面定义过
# f[0]元素也是在tape里面定义的，因此也可以求导
dy_dx = g.gradient(f[0], [x, z])
print(dy_dx)
# f[1]元素也是在tape里面定义的，因此也可以求导
dy_dx = g.gradient(f[1], [x, z])
print(dy_dx)
# 如果分子为列表，则对x求导的结果为列表里每个元素求导结果之和，如果某个元素不可导相当于为0, 除非所有元素都不可导则为None
dy_dx = g.gradient(f, [x, z, j])
print(dy_dx)
# 在tape外面定义的op是无法求导的，即便用到的东西全是在tape里面定义的
dd = tf.stack(f)
dy_dx = g.gradient(dd, [x, z])
print(dy_dx)
# 与list求导类似，对x求导结果是shape为(2,)的tensor的所有元素求导的结果之和
dy_dx = g.gradient(ff, [x, z])
print(dy_dx)
# 与list求导类似，对x求导结果是shape为(3,2)的tensor的所有元素求导的结果之和
dy_dx = g.gradient(ll, [x, z])
print(dy_dx)
# 在tape外面定义的任何操作是无法求导的，即便用到的东西全是在tape里面定义的
dy_dx = g.gradient(k / z, [x, z])
print(dy_dx)
# 自己推导一下
dy_dx = g.gradient(kz, [x, z])
print(dy_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]
[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>, None]
[None, None]
[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=27.0>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>]
[None, None]
[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=-0.33333334>]


In [18]:
# 二阶导数的例子
x = tf.constant(5.0)
with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)  # dy_dx = 2 * x
d2y_dx2 = g.gradient(dy_dx, x) # d2y_dx2 = 2
dy_dx2 = g.gradient(y, x)  # dy_dx = 2 * x
print(dy_dx)
print(dy_dx2)
print(d2y_dx2)

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


In [12]:
# 如果不用两个tape的话
x = tf.Variable(5.0)
with tf.GradientTape(persistent=True) as g:
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

# 第一个grad dy_dx现在对于第二个tape来说，就是一个constant, 无法求导
with tf.GradientTape(persistent=True) as gg:
  dd = dy_dx * 10
d2y_dx2 = gg.gradient(dd, [x, dy_dx])
print(d2y_dx2)

tf.Tensor(10.0, shape=(), dtype=float32)
[None, None]


In [17]:
x = tf.Variable(2.0)
f = []
with tf.GradientTape(persistent=True) as g:
  z = x * 3
  # 注意这里必须返回一个同名变量，否则相当于不起效果
  z = tf.stop_gradient(z)
  f.append(x * z)

# 求导的分子，必须是在tape里面定义过
# f[0]元素也是在tape里面定义的，因此也可以求导，效果同上
dy_dx = g.gradient(f[0], [x, z])
print(dy_dx)

# 对于第二个tape来说，dy_dx是一个shape为(2,)的constant, 它乘以变量x可以求导了
with tf.GradientTape(persistent=True) as g:
  ff = dy_dx * x
dy_dx = g.gradient(ff, [x, z])
print(dy_dx)

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>, None]


In [10]:
# 求导是计算的当前值上的导数，可以通过assign改变当前值
x = tf.Variable(5.0)
with tf.GradientTape() as g:
  x.assign(10.0)
  x.assign(20.0)
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(40.0, shape=(), dtype=float32)


In [3]:
'''
例3： loss和metrics在底层是如何计算的
'''
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

    def get_config(self):
        config = {'rate': self.rate}
        base_config = super(ActivityRegularizationLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class CustomModel(keras.Model):
  def __init__(self, *args, **kargs):
      super(CustomModel, self).__init__(*args, **kargs)
      self.log_vars= self.add_weight(name='log_var',
                                     shape=(1,),
                                     initializer=Constant(0.0),
                                     trainable=True)

  def compile(self, optimizer, loss, metrics, loss_weights):
    # 如果super.compile的时候指定了loss，
    #    self.compiled_loss执行后self.compiled_loss就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    # 如果super.compile的时候指定了metric,
    #    self.compiled_metric.update_state的时候self.compiled_metric就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    super(CustomModel, self).compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights, metrics=metrics)
    # 下面两句执行后，self.metrics就会添加对应的metric, 但是他们不属于self.compiled_metric
    self.mae_metric0 = keras.metrics.MeanSquaredError(name="my_mse0")
    self.mae_metric1 = keras.metrics.MeanSquaredError(name="my_mse1")


  def train_step(self, data):
    x, y, weight = data
    with tf.GradientTape() as tape:
      tf.print('lr', self.optimizer.lr)
      # 前向传播后产生模型中所有regularization、add_loss方法产生的loss，结果作为list保存在self.losses里面
      # 每个元素都是一个scalar
      #
      # 模型实例compile的时候指定的loss，是用于output产生的y_pred与y进行损失计算的，但是这些结果都不含在self.losses里面。
      # 我们在compile的时候传入了loss={'a': "mse", 'b': "mse"}，
      # self.compiled_loss调用结束后，self.compiled_loss变成含3个metic的list，分别对应了a的loss， b的loss, 和总体的loss
      # compiled_loss调用的时候会对数据结构做归一化，比如y_pred是含两个output的list， sample_weight是一个
      # 含有两个vector的字典或者sample_weight就是一个vector，最终都会被归一化成含两个weight vector的list。
      # 但是如果sample_weight是一个含有三个vector的字典或者list，只有前两个会被使用。
      # compiled_loss调用结束返回的是一个scalar：上述所有loss的和（regularization_losses=self.losses的时候）
      # 下面我们手动验证一下
      y_pred = self(x, training=True)
      mse_loss0 = keras.losses.mean_squared_error(y['a'], y_pred['a']) # (None,)
      mse_loss1 = keras.losses.mean_squared_error(y['b'], y_pred['b']) # (None,)
      tf.print('self.losses', self.losses,
               # 默认batch_size是32, 而我们样本只有3个，所以一个batch就是一个epoch，
               # 而每个epoch的metric都会被自动重置，所以
               # 一个batch的loss0应该和mse0_metric结果一样， loss1应该和mse1_metric结果一样
               'loss0', tf.reduce_mean(mse_loss0), 'loss1', tf.reduce_mean(mse_loss1),
               'my_total', tf.reduce_mean(1 * mse_loss0 * weight['a'] + 10 * mse_loss1 * weight['b']))
      loss = self.compiled_loss(y, y_pred, sample_weight=weight, regularization_losses=self.losses)
      # total = my_total + self.losses(也就是compiles_loss里面通过regularization_losses来进行操作)
      tf.print('total', loss)
      loss *= self.log_vars
      # 最终train_loop自动print出来的的loss不是这个，
      # return {m.name: m.result() for m in self.metrics} 才是决定train_loop print什么内容的
      # 注意train_loop 自动print的每个输出的loss是加权的，比如keras.losses.mean_squared_error(y['a'], y_pred['a'], weight['a'])
      tf.print('total——after', loss)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # 模型实例compile的时候如果指定了metric，则这个时候self.compiled_metrics会变成对应的metrics list，
    # 但是def compile里面自己定义的self.mae_metric0和self.mae_metric1不包含在self.compiled_metrics里面
    self.compiled_metrics.update_state(y, y_pred, sample_weight=weight)
    # 因此这里需要将自己定义的self.mae_metric0和self.mae_metric1 update
    self.mae_metric0.update_state(y['a'], y_pred['a'])
    self.mae_metric1.update_state(y['b'], y_pred['b'])
    tf.print('mse0_metric', self.mae_metric0.result(), 'mse1_metric', self.mae_metric1.result())
    # 上面已经讲过，self.compiled_loss会变成metric list去跟踪loss, 但是我们不需要去管他们，他们是自动的
    # self.metrics包含三部分内容，
    #    一部分是self.compiled_loss自己生成metrics,
    #    一部分是self.compiled_metrics生成的metrics
    #    一部分是def compile里面自己添加的metrics(他们虽然在def compile里面定义，但是不输入self.compiled_metrics)
    return {m.name: m.result() for m in self.metrics}

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
outputs1 = keras.layers.Dense(1)(outputs)
outputs2 = keras.layers.Dense(1)(outputs)
model = CustomModel(inputs, {'a': outputs1, 'b': outputs2})
model.compile(optimizer="adam",
              metrics='mse',
              loss={'a': "mse", 'b': "mse"},
              loss_weights={'a': 1.0, 'b': 10.0})

x = np.random.random((3, 3))
y = {'a': np.random.random((3, 1)), 'b': np.random.random((3, 1))}
weight = {'a': np.array([1.0,0.0,0.0]),
          'b': np.array([0.0,1.0,1.0])}
model.fit(x,
          y,
          epochs=5,
          sample_weight=weight)

Epoch 1/5
lr 0.001
self.losses [0.0429790951] loss0 0.310979337 loss1 0.89723891 my_total 7.60930872
total 7.65228844
total——after [0]
mse0_metric 0.310979337 mse1_metric 0.89723891
1/1 [==============================] - 0s 2ms/step - loss: 7.6523 - dense_2_loss: 0.0986 - dense_3_loss: 0.7511 - dense_2_mse: 0.3110 - dense_3_mse: 0.8972 - my_mse0: 0.3110 - my_mse1: 0.8972
Epoch 2/5
lr 0.001
self.losses [0.0429791026] loss0 0.310979366 loss1 0.89723891 my_total 7.60930872
total 7.65228796
total——after [-0.00765228458]
mse0_metric 0.310979366 mse1_metric 0.89723891
1/1 [==============================] - 0s 1ms/step - loss: 7.6523 - dense_2_loss: 0.0986 - dense_3_loss: 0.7511 - dense_2_mse: 0.3110 - dense_3_mse: 0.8972 - my_mse0: 0.3110 - my_mse1: 0.8972

In [14]:
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

    def get_config(self):
        config = {'rate': self.rate}
        base_config = super(ActivityRegularizationLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class CustomModel(keras.Model):
  def __init__(self, *args, **kargs):
      super(CustomModel, self).__init__(*args, **kargs)
      self.log_vars= self.add_weight(name='log_var',
                                     shape=(1,),
                                     initializer=Constant(0.0),
                                     trainable=True)

  def compile(self, optimizer, loss, metrics, loss_weights):
    # 如果super.compile的时候指定了loss，
    #    self.compiled_loss执行后self.compiled_loss就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    # 如果super.compile的时候指定了metric,
    #    self.compiled_metric.update_state的时候self.compiled_metric就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    super(CustomModel, self).compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights, metrics=metrics)
    # 下面两句执行后，self.metrics就会添加对应的metric, 但是他们不属于self.compiled_metric
    self.mae_metric0 = keras.metrics.MeanSquaredError(name="my_mse0")
    self.mae_metric1 = keras.metrics.MeanSquaredError(name="my_mse1")


  def train_step(self, data):
    x, y, weight = data
    with tf.GradientTape() as tape:
      tf.print('lr', self.optimizer.lr)
      # 前向传播后产生模型中所有regularization、add_loss方法产生的loss，结果作为list保存在self.losses里面
      # 每个元素都是一个scalar
      #
      # 模型实例compile的时候指定的loss，是用于output产生的y_pred与y进行损失计算的，但是这些结果都不含在self.losses里面。
      # 我们在compile的时候传入了loss={'a': "mse", 'b': "mse"}，
      # self.compiled_loss调用结束后，self.compiled_loss变成含3个metic的list，分别对应了a的loss， b的loss, 和总体的loss
      # compiled_loss调用的时候会对数据结构做归一化，比如y_pred是含两个output的list， sample_weight是一个
      # 含有两个vector的字典或者sample_weight就是一个vector，最终都会被归一化成含两个weight vector的list。
      # 但是如果sample_weight是一个含有三个vector的字典或者list，只有前两个会被使用。
      # compiled_loss调用结束返回的是一个scalar：上述所有loss的和（regularization_losses=self.losses的时候）
      # 下面我们手动验证一下
      y_pred = self(x, training=True)
      mse_loss0 = keras.losses.MeanSquaredError()(y['a'], y_pred['a'], weight['a']) # scalar
      mse_loss1 = keras.losses.MeanSquaredError()(y['b'], y_pred['b'], weight['b']) # scalar
      tf.print('self.losses', self.losses,
               # 默认batch_size是32, 而我们样本只有3个，所以一个batch就是一个epoch，
               # 而每个epoch的metric都会被自动重置，所以
               # 一个batch的loss0应该和mse0_metric结果一样， loss1应该和mse1_metric结果一样
               'loss0', mse_loss0, 'loss1', mse_loss1,
               'my_total', 1 * mse_loss0 + 10 * mse_loss1)
      loss = self.compiled_loss(y, y_pred, sample_weight=weight, regularization_losses=self.losses)
      # total = my_total + self.losses(也就是compiles_loss里面通过regularization_losses来进行操作)
      tf.print('total', loss)
      loss *= self.log_vars
      # 最终train_loop自动print出来的的loss不是这个，
      # return {m.name: m.result() for m in self.metrics} 才是决定train_loop print什么内容的
      # 注意train_loop 自动print的每个输出的loss是加权的，比如keras.losses.mean_squared_error(y['a'], y_pred['a'], weight['a'])
      tf.print('total——after', loss)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # 模型实例compile的时候如果指定了metric，则这个时候self.compiled_metrics会变成对应的metrics list，
    # 但是def compile里面自己定义的self.mae_metric0和self.mae_metric1不包含在self.compiled_metrics里面
    self.compiled_metrics.update_state(y, y_pred, sample_weight=weight)
    # 因此这里需要将自己定义的self.mae_metric0和self.mae_metric1 update
    self.mae_metric0.update_state(y['a'], y_pred['a'])
    self.mae_metric1.update_state(y['b'], y_pred['b'])
    tf.print('mse0_metric', self.mae_metric0.result(), 'mse1_metric', self.mae_metric1.result())
    # 上面已经讲过，self.compiled_loss会变成metric list去跟踪loss, 但是我们不需要去管他们，他们是自动的
    # self.metrics包含三部分内容，
    #    一部分是self.compiled_loss自己生成metrics,
    #    一部分是self.compiled_metrics生成的metrics
    #    一部分是def compile里面自己添加的metrics(他们虽然在def compile里面定义，但是不输入self.compiled_metrics)
    return {m.name: m.result() for m in self.metrics}

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
outputs1 = keras.layers.Dense(1)(outputs)
outputs2 = keras.layers.Dense(1)(outputs)
model = CustomModel(inputs, {'a': outputs1, 'b': outputs2})
model.compile(optimizer="adam",
              metrics='mse',
              loss={'a': "mse", 'b': "mse"},
              loss_weights={'a': 1.0, 'b': 10.0})

x = np.random.random((3, 3))
y = {'a': np.random.random((3, 1)), 'b': np.random.random((3, 1))}
weight = {'a': np.array([1.0,0.0,0.0]),
          'b': np.array([0.0,1.0,1.0])}
model.fit(x,
          y,
          epochs=5,
          sample_weight=weight)

Epoch 1/5
lr 0.001
self.losses [0.0497862957] loss0 0.0588801168 loss1 0.001637607 my_total 0.0752561837
total 0.125042483
total——after [0]
mse0_metric 0.111876391 mse1_metric 0.0717143491
1/1 [==============================] - 0s 2ms/step - loss: 0.1250 - dense_18_loss: 0.0589 - dense_19_loss: 0.0016 - dense_18_mse: 0.1119 - dense_19_mse: 0.0717 - my_mse0: 0.1119 - my_mse1: 0.0717
Epoch 2/5
lr 0.001
self.losses [0.0497862957] loss0 0.0588801168 loss1 0.001637607 my_total 0.0752561837
total 0.125042483
total——after [-0.000125039311]
mse0_metric 0.111876391 mse1_metric 0.0717143491
1/1 [==============================] - 0s 1ms/step - loss: 0.1250 - dense_18_loss: 0.0589 - dense_19_loss: 0.0016 - dense_18_mse: 0.1119 - dense_19_mse: 0.0717 - my_mse0: 0.1119 - my_mse1: 0.0717

In [17]:
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

    def get_config(self):
        config = {'rate': self.rate}
        base_config = super(ActivityRegularizationLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class CustomModel(keras.Model):
  def __init__(self, *args, **kargs):
      super(CustomModel, self).__init__(*args, **kargs)
      self.log_vars= self.add_weight(name='log_var',
                                     shape=(1,),
                                     initializer=Constant(0.0),
                                     trainable=True)

  def compile(self, optimizer, loss, metrics, loss_weights):
    # 如果super.compile的时候指定了loss，
    #    self.compiled_loss执行后self.compiled_loss就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    # 如果super.compile的时候指定了metric,
    #    self.compiled_metric.update_state的时候self.compiled_metric就变成了对应的metric的list
    #    同时self.metrics也会添加对应的metric
    super(CustomModel, self).compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights, metrics=metrics)
    # 下面两句执行后，self.metrics就会添加对应的metric, 但是他们不属于self.compiled_metric
    self.mae_metric0 = keras.metrics.MeanSquaredError(name="my_mse0")
    self.mae_metric1 = keras.metrics.MeanSquaredError(name="my_mse1")


  def train_step(self, data):
    x, y, weight = data
    with tf.GradientTape() as tape:
      tf.print('lr', self.optimizer.lr)
      # 前向传播后产生模型中所有regularization、add_loss方法产生的loss，结果作为list保存在self.losses里面
      # 每个元素都是一个scalar
      #
      # 模型实例compile的时候指定的loss，是用于output产生的y_pred与y进行损失计算的，但是这些结果都不含在self.losses里面。
      # 我们在compile的时候传入了loss={'a': "mse", 'b': "mse"}，
      # self.compiled_loss调用结束后，self.compiled_loss变成含3个metic的list，分别对应了a的loss， b的loss, 和总体的loss
      # compiled_loss调用的时候会对数据结构做归一化，比如y_pred是含两个output的list， sample_weight是一个
      # 含有两个vector的字典或者sample_weight就是一个vector，最终都会被归一化成含两个weight vector的list。
      # 但是如果sample_weight是一个含有三个vector的字典或者list，只有前两个会被使用。
      # compiled_loss调用结束返回的是一个scalar：上述所有loss的和（regularization_losses=self.losses的时候）
      # 下面我们手动验证一下
      y_pred = self(x, training=True)
      mse_loss0 = keras.losses.mean_squared_error(y['a'], y_pred['a']) # (None,)
      mse_loss1 = keras.losses.mean_squared_error(y['b'], y_pred['b']) # (None,)
      tf.print('self.losses', self.losses,
               # 默认batch_size是32, 而我们样本只有3个，所以一个batch就是一个epoch，
               # 而每个epoch的metric都会被自动重置，所以
               # 一个batch的loss0应该和mse0_metric结果一样， loss1应该和mse1_metric结果一样
               'loss0', tf.reduce_mean(mse_loss0), 'loss1', tf.reduce_mean(mse_loss1),
               'my_total', tf.reduce_mean(1 * mse_loss0 * weight['a'] / tf.reduce_sum(weight['a']) +
                                          10 * mse_loss1 * weight['b'] / tf.reduce_sum(weight['b'])))
      loss = self.compiled_loss(y, y_pred, sample_weight=weight, regularization_losses=self.losses)
      # total = my_total + self.losses(也就是compiles_loss里面通过regularization_losses来进行操作)
      tf.print('total', loss)
      loss *= self.log_vars
      # 最终train_loop自动print出来的的loss不是这个，
      # return {m.name: m.result() for m in self.metrics} 才是决定train_loop print什么内容的
      # 注意train_loop 自动print的每个输出的loss是加权的，比如keras.losses.mean_squared_error(y['a'], y_pred['a'], weight['a'])
      tf.print('total——after', loss)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # 模型实例compile的时候如果指定了metric，则这个时候self.compiled_metrics会变成对应的metrics list，
    # 但是def compile里面自己定义的self.mae_metric0和self.mae_metric1不包含在self.compiled_metrics里面
    self.compiled_metrics.update_state(y, y_pred, sample_weight=weight)
    # 因此这里需要将自己定义的self.mae_metric0和self.mae_metric1 update
    self.mae_metric0.update_state(y['a'], y_pred['a'])
    self.mae_metric1.update_state(y['b'], y_pred['b'])
    tf.print('mse0_metric', self.mae_metric0.result(), 'mse1_metric', self.mae_metric1.result())
    # 上面已经讲过，self.compiled_loss会变成metric list去跟踪loss, 但是我们不需要去管他们，他们是自动的
    # self.metrics包含三部分内容，
    #    一部分是self.compiled_loss自己生成metrics,
    #    一部分是self.compiled_metrics生成的metrics
    #    一部分是def compile里面自己添加的metrics(他们虽然在def compile里面定义，但是不输入self.compiled_metrics)
    return {m.name: m.result() for m in self.metrics}

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
outputs1 = keras.layers.Dense(1)(outputs)
outputs2 = keras.layers.Dense(1)(outputs)
model = CustomModel(inputs, {'a': outputs1, 'b': outputs2})
model.compile(optimizer="adam",
              metrics='mse',
              loss={'a': "mse", 'b': "mse"},
              loss_weights={'a': 1.0, 'b': 10.0})

x = np.random.random((3, 3))
y = {'a': np.random.random((3, 1)), 'b': np.random.random((3, 1))}
weight = {'a': np.array([1.0,0.0,0.0]),
          'b': np.array([0.0,1.0,1.0])}
model.fit(x,
          y,
          epochs=5,
          sample_weight=weight)

Epoch 1/5
lr 0.001
self.losses [0.0504987575] loss0 0.310474128 loss1 0.996884346 my_total 3.65924668
total 7.32167816
total——after [0]
mse0_metric 0.310474128 mse1_metric 0.996884346
1/1 [==============================] - 0s 2ms/step - loss: 7.3217 - dense_20_loss: 0.0473 - dense_21_loss: 0.7224 - dense_20_mse: 0.3105 - dense_21_mse: 0.9969 - my_mse0: 0.3105 - my_mse1: 0.9969
Epoch 2/5
lr 0.001
self.losses [0.0504987612] loss0 0.310474128 loss1 0.996884346 my_total 3.65924668
total 7.32167816
total——after [-0.00732167531]
mse0_metric 0.310474128 mse1_metric 0.996884346
1/1 [==============================] - 0s 2ms/step - loss: 7.3217 - dense_20_loss: 0.0473 - dense_21_loss: 0.7224 - dense_20_mse: 0.3105 - dense_21_mse: 0.9969 - my_mse0: 0.3105 - my_mse1: 0.9969

In [28]:
'''
例4：自定义模型的复原
'''
model.save_weights('muy')
# 模型的权重是包含两个自定义的metric的,不过没关系，重新训练的时候都会被重置的
print(model.weights)

[<tf.Variable 'dense_22/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.71009797],
       [0.12378993],
       [0.55186844]], dtype=float32)>, <tf.Variable 'dense_22/bias:0' shape=(1,) dtype=float32, numpy=array([0.00328385], dtype=float32)>, <tf.Variable 'dense_23/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.6119768 ],
       [0.5554451 ],
       [0.01648403]], dtype=float32)>, <tf.Variable 'dense_23/bias:0' shape=(1,) dtype=float32, numpy=array([-0.00334061], dtype=float32)>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=1.0930122>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=0.310604>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>, <tf.Variable 'log_var:0' shape=(1,) dtype=float32, numpy=array([-0.00500159], dtype=float32)>]


In [29]:
model2 = CustomModel(inputs, {'a': outputs1, 'b': outputs2})
# 此时权重是不包含metric的（上面已经讲过，自定义的metric在def compile里面才会生成，因此这个时候是没有他们的权重的）
# 但是这个时候有log_var的权重，这个在__ini__里面定义的，所以只要实例化就可以有的，不过是初始值0
print(model2.weights)

[<tf.Variable 'dense_22/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.71009797],
       [0.12378993],
       [0.55186844]], dtype=float32)>, <tf.Variable 'dense_22/bias:0' shape=(1,) dtype=float32, numpy=array([0.00328385], dtype=float32)>, <tf.Variable 'dense_23/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.6119768 ],
       [0.5554451 ],
       [0.01648403]], dtype=float32)>, <tf.Variable 'dense_23/bias:0' shape=(1,) dtype=float32, numpy=array([-0.00334061], dtype=float32)>, <tf.Variable 'log_var:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [31]:
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.6),
               metrics='mse',
               loss={'a': "mse", 'b': "mse"},
               loss_weights={'a': 1.0, 'b': 10.0})
# 这个时候已经compile过了，可以看到模型产生了自定义的metric的权重，不过是初始值0
print(model2.weights)

[<tf.Variable 'dense_22/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.71009797],
       [0.12378993],
       [0.55186844]], dtype=float32)>, <tf.Variable 'dense_22/bias:0' shape=(1,) dtype=float32, numpy=array([0.00328385], dtype=float32)>, <tf.Variable 'dense_23/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.6119768 ],
       [0.5554451 ],
       [0.01648403]], dtype=float32)>, <tf.Variable 'dense_23/bias:0' shape=(1,) dtype=float32, numpy=array([-0.00334061], dtype=float32)>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'log_var:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [33]:
model2.train_on_batch(x[:1],
                      {name: value[:1] for name, value in y.items()},
                      sample_weight={name: value[:1] for name, value in weight.items()})
model2.load_weights('muy')
# 这个时候，无论是自定义的metric的权重，log_var的权重，optimizers的权重，全部都复原了！
print(model2.weights)

lr 0.6
self.losses [0.00785260927] loss0 0.216828778 loss1 0.25319913 my_total 25.5367413
total 25.5445938
total——after [0]
mse0_metric 0.216828778 mse1_metric 0.25319913
[<tf.Variable 'dense_22/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.71009797],
       [0.12378993],
       [0.55186844]], dtype=float32)>, <tf.Variable 'dense_22/bias:0' shape=(1,) dtype=float32, numpy=array([0.00328385], dtype=float32)>, <tf.Variable 'dense_23/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.6119768 ],
       [0.5554451 ],
       [0.01648403]], dtype=float32)>, <tf.Variable 'dense_23/bias:0' shape=(1,) dtype=float32, numpy=array([-0.00334061], dtype=float32)>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=1.0930122>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>, <tf.Variable 'total:0' shape=() dtype=float32, numpy=0.310604>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>, <tf.Variable 'log_var:0' shape=(1,) dtype=float32, numpy=array([-0.00500159], dtyp

In [34]:
# 这个时候就相当于恢复训练了，可以看到连学习率都复原了，而不是本次初始化时的0.6
model2.fit(x,
           y,
           epochs=5,
           sample_weight=weight)

Epoch 1/5
lr 0.001
self.losses [0.0397862755] loss0 0.365132928 loss1 0.104212619 my_total 10.786396
total 10.8261824
total——after [-0.0541481413]
mse0_metric 0.365132928 mse1_metric 0.104212619
1/1 [==============================] - 0s 0s/step - loss: 10.8262 - dense_22_loss: 0.3651 - dense_23_loss: 1.0421 - dense_22_mse: 0.3651 - dense_23_mse: 0.1042 - my_mse0: 0.3651 - my_mse1: 0.1042
Epoch 2/5
lr 0.001
self.losses [0.0397862718] loss0 0.365959436 loss1 0.104917973 my_total 10.8577576
total 10.897542
total——after [-0.0654188097]
mse0_metric 0.365959436 mse1_metric 0.104917973
1/1 [==============================] - 0s 2ms/step - loss: 10.8975 - dense_22_loss: 0.3660 - dense_23_loss: 1.0492 - dense_22_mse: 0.3660 - dense_23_mse: 0.1049 - my_mse0: 0.3660 - my_mse1: 0.1049

In [ ]:
'''
例5：自定义模型可以用model.save方法保存吗？
目前看来，只要是Model(input, ouput)这种形式的都不行
'''